In [ ]:
# Task 3: Correlation Analysis - Interactive Notebook

import sys
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from task3_correlation_analysis import CorrelationAnalyzer

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Task 3: Correlation Analysis between News Sentiment and Stock Returns")
# Section 1: Setup and Data Loading
print("Loading and preparing data...")

# Initialize the correlation analyzer
analyzer = CorrelationAnalyzer(
    news_data_path='../data/processed/news_data.csv',
    stock_data_path='../data/processed/stock_data.csv'
)

# Load the data
analyzer.load_data()

# Display basic information about the datasets
print("\nNews Data Overview:")
print(f"Shape: {analyzer.news_data.shape}")
print("\nFirst few rows:")
display(analyzer.news_data.head())

print("\nStock Data Overview:")
print(f"Shape: {analyzer.stock_data.shape}")
print("\nFirst few rows:")
display(analyzer.stock_data.head())
# Section 2: Explore News Data Statistics
print("News Data Statistics:")

# Basic stats about news data
print(f"Total news articles: {len(analyzer.news_data)}")
print(f"Date range: {analyzer.news_data['publication_date'].min()} to {analyzer.news_data['publication_date'].max()}")
print(f"Number of unique dates: {analyzer.news_data['publication_date'].nunique()}")

# Articles per day
articles_per_day = analyzer.news_data.groupby('publication_date').size()
print(f"\nAverage articles per day: {articles_per_day.mean():.2f}")
print(f"Maximum articles in a day: {articles_per_day.max()}")
print(f"Minimum articles in a day: {articles_per_day.min()}")

# Plot articles per day
plt.figure(figsize=(12, 6))
articles_per_day.plot(kind='line', title='Number of News Articles Per Day')
plt.xlabel('Date')
plt.ylabel('Number of Articles')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()
# Section 3: Explore Stock Data Statistics
print("Stock Data Statistics:")

# Basic stats about stock data
print(f"Total trading days: {len(analyzer.stock_data)}")
print(f"Date range: {analyzer.stock_data['Date'].min()} to {analyzer.stock_data['Date'].max()}")

# Stock price statistics
print(f"\nStock Price Statistics:")
print(f"Average Close Price: ${analyzer.stock_data['Close'].mean():.2f}")
print(f"High: ${analyzer.stock_data['High'].max():.2f}")
print(f"Low: ${analyzer.stock_data['Low'].min():.2f}")

# Plot stock price movement
plt.figure(figsize=(12, 8))

plt.subplot(2, 1, 1)
plt.plot(analyzer.stock_data['Date'], analyzer.stock_data['Close'], linewidth=2)
plt.title('Stock Price Movement')
plt.ylabel('Price ($)')
plt.grid(True, alpha=0.3)

plt.subplot(2, 1, 2)
plt.plot(analyzer.stock_data['Date'], analyzer.stock_data['Volume'], alpha=0.7)
plt.title('Trading Volume')
plt.xlabel('Date')
plt.ylabel('Volume')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()
# Section 4: Perform Sentiment Analysis
print("Performing Sentiment Analysis on News Headlines...")

# Perform sentiment analysis
sentiment_data = analyzer.analyze_sentiment()

# Display sentiment results
print("\nSentiment Analysis Results:")
print(f"Analyzed {len(analyzer.news_data)} news articles")
print(f"Generated sentiment for {len(sentiment_data)} days")

print("\nSentiment Statistics:")
print(f"Average Daily Sentiment: {sentiment_data['avg_sentiment'].mean():.4f}")
print(f"Sentiment Std Dev: {sentiment_data['avg_sentiment'].std():.4f}")
print(f"Most Positive Day: {sentiment_data['avg_sentiment'].max():.4f}")
print(f"Most Negative Day: {sentiment_data['avg_sentiment'].min():.4f}")

# Display sentiment data
display(sentiment_data.head(10))# Section 5: Visualize Sentiment Analysis Results
print("Visualizing Sentiment Analysis Results...")

# Create comprehensive sentiment visualizations
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Plot 1: Daily average sentiment over time
axes[0, 0].plot(sentiment_data['publication_date'], sentiment_data['avg_sentiment'], 
                color='blue', linewidth=2, alpha=0.8)
axes[0, 0].set_title('Daily Average Sentiment Over Time')
axes[0, 0].set_xlabel('Date')
axes[0, 0].set_ylabel('Sentiment Score')
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].axhline(y=0, color='red', linestyle='--', alpha=0.5)

# Plot 2: Distribution of sentiment scores
axes[0, 1].hist(sentiment_data['avg_sentiment'], bins=20, alpha=0.7, 
                color='green', edgecolor='black')
axes[0, 1].set_title('Distribution of Daily Average Sentiment')
axes[0, 1].set_xlabel('Sentiment Score')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: Articles per day vs sentiment
axes[1, 0].scatter(sentiment_data['article_count'], sentiment_data['avg_sentiment'], 
                   alpha=0.6, color='purple')
axes[1, 0].set_title('Articles per Day vs Sentiment')
axes[1, 0].set_xlabel('Number of Articles')
axes[1, 0].set_ylabel('Average Sentiment')
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: Sentiment volatility
axes[1, 1].plot(sentiment_data['publication_date'], sentiment_data['sentiment_std'], 
                color='orange', alpha=0.8)
axes[1, 1].set_title('Sentiment Volatility (Standard Deviation)')
axes[1, 1].set_xlabel('Date')
axes[1, 1].set_ylabel('Sentiment Std Dev')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Additional: Sentiment heatmap by day of week
sentiment_data['day_of_week'] = sentiment_data['publication_date'].dt.day_name()
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
sentiment_by_day = sentiment_data.groupby('day_of_week')['avg_sentiment'].mean().reindex(day_order)

plt.figure(figsize=(10, 6))
plt.bar(sentiment_by_day.index, sentiment_by_day.values, color='skyblue', alpha=0.7)
plt.title('Average Sentiment by Day of Week')
plt.xlabel('Day of Week')
plt.ylabel('Average Sentiment Score')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()
# Section 6: Compute Daily Returns and Prepare for Correlation
print("Computing Daily Returns and Merging Data...")

# Compute daily returns
returns_data = analyzer.compute_daily_returns()

print("\nReturns Statistics:")
print(f"Average Daily Return: {returns_data['daily_return'].mean():.4f}%")
print(f"Return Std Dev: {returns_data['daily_return'].std():.4f}%")
print(f"Maximum Daily Return: {returns_data['daily_return'].max():.4f}%")
print(f"Minimum Daily Return: {returns_data['daily_return'].min():.4f}%")

# Display returns data
display(returns_data.head(10))

# Plot returns distribution
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(returns_data['daily_return'], bins=30, alpha=0.7, color='blue', edgecolor='black')
plt.title('Distribution of Daily Returns')
plt.xlabel('Daily Return (%)')
plt.ylabel('Frequency')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(returns_data['Date'], returns_data['daily_return'], alpha=0.7)
plt.title('Daily Returns Over Time')
plt.xlabel('Date')
plt.ylabel('Daily Return (%)')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()
# Section 7: Merge Sentiment and Returns Data
print("Merging Sentiment and Returns Data...")

# Merge the datasets
merged_data = analyzer.merge_sentiment_returns()

print(f"Merged dataset shape: {merged_data.shape}")
print(f"Number of matching days with both sentiment and returns: {len(merged_data)}")

# Display merged data
print("\nMerged Data Sample:")
display(merged_data.head(10))

# Basic statistics of merged data
print("\nMerged Dataset Statistics:")
print(f"Date range: {merged_data['publication_date'].min()} to {merged_data['publication_date'].max()}")
print(f"Average sentiment: {merged_data['avg_sentiment'].mean():.4f}")
print(f"Average daily return: {merged_data['daily_return'].mean():.4f}%")
print(f"Correlation between sentiment and returns: {merged_data['avg_sentiment'].corr(merged_data['daily_return']):.4f}")
# Section 8: Perform Correlation Analysis
print("Performing Correlation Analysis...")

# Calculate correlation coefficients
correlation_results = analyzer.calculate_correlation(merged_data)

# Display correlation results
print("\n" + "="*50)
print("CORRELATION ANALYSIS RESULTS")
print("="*50)

print(f"\nSample Size: {correlation_results['sample_size']} days")

print(f"\nPearson Correlation Coefficients:")
print(f"Sentiment vs Daily Returns: {correlation_results['daily_return_correlation']:.4f}")
print(f"P-value: {correlation_results['daily_return_p_value']:.4f}")

print(f"Sentiment vs Log Returns: {correlation_results['log_return_correlation']:.4f}")
print(f"P-value: {correlation_results['log_return_p_value']:.4f}")

print(f"Sentiment vs Next Day Returns: {correlation_results['lagged_correlation']:.4f}")
print(f"P-value: {correlation_results['lagged_p_value']:.4f}")

# Interpret the results
print(f"\nINTERPRETATION:")
corr_value = correlation_results['daily_return_correlation']
p_value = correlation_results['daily_return_p_value']

# Correlation strength
if abs(corr_value) < 0.1:
    strength = "negligible"
elif abs(corr_value) < 0.3:
    strength = "weak"
elif abs(corr_value) < 0.5:
    strength = "moderate"
else:
    strength = "strong"

direction = "positive" if corr_value > 0 else "negative"

print(f"The correlation between news sentiment and stock returns is {strength} and {direction}.")

# Statistical significance
if p_value < 0.05:
    print("The correlation is statistically significant (p < 0.05).")
else:
    print("The correlation is not statistically significant (p >= 0.05).")

# Additional correlation matrix
print(f"\nAdditional Correlations:")
correlation_matrix = merged_data[['avg_sentiment', 'daily_return', 'log_return', 'article_count', 'Volume']].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, fmt='.3f', linewidths=0.5)
plt.title('Correlation Matrix: Sentiment, Returns, and Market Factors')
plt.tight_layout()
plt.show()
# Section 9: Comprehensive Visualization
print("Generating Comprehensive Visualizations...")

# Generate all visualizations
analyzer.visualize_correlation(merged_data, save_path='../results/task3_correlation_analysis.png')

# Additional custom visualizations
print("\nGenerating Additional Custom Visualizations...")

# 1. Rolling correlation over time
window_size = 30  # 30-day rolling window
merged_data_sorted = merged_data.sort_values('publication_date').copy()
merged_data_sorted['rolling_corr'] = merged_data_sorted['avg_sentiment'].rolling(window=window_size).corr(merged_data_sorted['daily_return'])

plt.figure(figsize=(12, 8))

plt.subplot(2, 1, 1)
plt.plot(merged_data_sorted['publication_date'], merged_data_sorted['rolling_corr'], 
         color='purple', linewidth=2)
plt.axhline(y=0, color='red', linestyle='--', alpha=0.5)
plt.title(f'{window_size}-Day Rolling Correlation Between Sentiment and Returns')
plt.ylabel('Rolling Correlation')
plt.grid(True, alpha=0.3)

plt.subplot(2, 1, 2)
plt.plot(merged_data_sorted['publication_date'], merged_data_sorted['avg_sentiment'], 
         label='Sentiment', alpha=0.7)
plt.plot(merged_data_sorted['publication_date'], merged_data_sorted['daily_return'], 
         label='Returns', alpha=0.7)
plt.title('Sentiment and Returns Over Time')
plt.xlabel('Date')
plt.ylabel('Values')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 2. Sentiment vs Returns by sentiment categories
merged_data['sentiment_category'] = pd.cut(merged_data['avg_sentiment'], 
                                          bins=[-1, -0.1, 0.1, 1], 
                                          labels=['Negative', 'Neutral', 'Positive'])

plt.figure(figsize=(10, 6))
sns.boxplot(data=merged_data, x='sentiment_category', y='daily_return')
plt.title('Daily Returns by Sentiment Category')
plt.xlabel('Sentiment Category')
plt.ylabel('Daily Return (%)')
plt.grid(True, alpha=0.3)
plt.show()

# 3. Scatter plot with regression line
plt.figure(figsize=(10, 6))
sns.regplot(data=merged_data, x='avg_sentiment', y='daily_return', 
            scatter_kws={'alpha':0.5}, line_kws={'color':'red'})
plt.title(f'Sentiment vs Daily Returns (Correlation: {correlation_results["daily_return_correlation"]:.3f})')
plt.xlabel('Average Daily Sentiment')
plt.ylabel('Daily Return (%)')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()
# Section 10: Advanced Analysis - Lagged Effects
print("Analyzing Lagged Effects...")

# Create lagged variables for analysis
lags = range(0, 6)  # Analyze lags from 0 to 5 days
lag_correlations = []

for lag in lags:
    if lag == 0:
        # Same day correlation
        corr, p_val = pearsonr(merged_data['avg_sentiment'], merged_data['daily_return'])
    else:
        # Create lagged returns (sentiment today vs returns in future)
        temp_data = merged_data.copy()
        temp_data[f'return_lag_{lag}'] = temp_data['daily_return'].shift(-lag)
        temp_data_lagged = temp_data.dropna()
        
        if len(temp_data_lagged) > 0:
            corr, p_val = pearsonr(temp_data_lagged['avg_sentiment'], 
                                 temp_data_lagged[f'return_lag_{lag}'])
        else:
            corr, p_val = (np.nan, np.nan)
    
    lag_correlations.append({
        'lag_days': lag,
        'correlation': corr,
        'p_value': p_val,
        'significant': p_val < 0.05 if not np.isnan(p_val) else False
    })

# Convert to DataFrame
lag_results = pd.DataFrame(lag_correlations)

print("Lagged Correlation Analysis:")
display(lag_results)

# Plot lagged correlations
plt.figure(figsize=(10, 6))
bars = plt.bar(lag_results['lag_days'], lag_results['correlation'], 
               color=['red' if sig else 'blue' for sig in lag_results['significant']],
               alpha=0.7)

plt.axhline(y=0, color='black', linestyle='-', alpha=0.3)
plt.title('Correlation Between Sentiment and Future Returns (Lagged Analysis)')
plt.xlabel('Lag (Days)')
plt.ylabel('Correlation Coefficient')
plt.xticks(lags)
plt.grid(True, alpha=0.3)

# Add value labels on bars
for bar, corr in zip(bars, lag_results['correlation']):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01 * (1 if bar.get_height() > 0 else -1),
             f'{corr:.3f}', ha='center', va='bottom' if bar.get_height() > 0 else 'top')

plt.tight_layout()
plt.show()
# Section 11: Save Results and Generate Final Report
print("Saving Results and Generating Final Report...")

# Save the merged dataset
merged_data.to_csv('../data/processed/sentiment_returns_merged.csv', index=False)
print("Saved merged dataset to: ../data/processed/sentiment_returns_merged.csv")

# Save correlation results
correlation_df = pd.DataFrame([correlation_results])
correlation_df.to_csv('../results/task3_correlation_results.csv', index=False)
print("Saved correlation results to: ../results/task3_correlation_results.csv")

# Save lag analysis results
lag_results.to_csv('../results/task3_lag_analysis.csv', index=False)
print("Saved lag analysis results to: ../results/task3_lag_analysis.csv")

# Generate final comprehensive report
print("\n" + "="*60)
print("TASK 3: FINAL CORRELATION ANALYSIS REPORT")
print("="*60)

print(f"\nDATA OVERVIEW:")
print(f"- News articles analyzed: {len(analyzer.news_data)}")
print(f"- Trading days analyzed: {len(analyzer.stock_data)}")
print(f"- Matching days with both sentiment and returns: {len(merged_data)}")
print(f"- Analysis period: {merged_data['publication_date'].min().strftime('%Y-%m-%d')} to {merged_data['publication_date'].max().strftime('%Y-%m-%d')}")

print(f"\nKEY FINDINGS:")
print(f"1. Primary Correlation (Same Day): {correlation_results['daily_return_correlation']:.4f}")
print(f"   Statistical Significance: {'Yes' if correlation_results['daily_return_p_value'] < 0.05 else 'No'}")

if not np.isnan(correlation_results['lagged_correlation']):
    print(f"2. Predictive Correlation (Next Day): {correlation_results['lagged_correlation']:.4f}")
    print(f"   Statistical Significance: {'Yes' if correlation_results['lagged_p_value'] < 0.05 else 'No'}")

print(f"\nINTERPRETATION:")
if abs(correlation_results['daily_return_correlation']) > 0.3 and correlation_results['daily_return_p_value'] < 0.05:
    print("✅ STRONG EVIDENCE of relationship between news sentiment and stock returns")
elif abs(correlation_results['daily_return_correlation']) > 0.1 and correlation_results['daily_return_p_value'] < 0.05:
    print("⚠️ MODERATE EVIDENCE of relationship between news sentiment and stock returns")
else:
    print("❕ WEAK or NO EVIDENCE of relationship between news sentiment and stock returns")

print(f"\nRECOMMENDATIONS:")
if correlation_results['daily_return_p_value'] < 0.05:
    if correlation_results['daily_return_correlation'] > 0:
        print("- Positive news sentiment tends to correlate with positive stock returns")
        print("- Consider incorporating sentiment analysis in trading strategies")
    else:
        print("- Negative news sentiment tends to correlate with negative stock returns")
        print("- Sentiment could be used as a contrarian indicator")
else:
    print("- No statistically significant relationship found")
    print("- News sentiment may not be a reliable indicator for this dataset/time period")

print(f"\nFILES GENERATED:")
print("1. ../data/processed/sentiment_returns_merged.csv - Merged dataset")
print("2. ../results/task3_correlation_results.csv - Correlation coefficients")
print("3. ../results/task3_lag_analysis.csv - Lagged analysis results")
print("4. ../results/task3_correlation_analysis.png - Comprehensive visualizations")

print(f"\nTask 3 Correlation Analysis Completed Successfully! 🎉")